<a href="https://colab.research.google.com/github/takeshitayy/openai-whisper/blob/main/openai_whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# whisperで音声ファイルをテキストに変換&

## 実行方法

1. ランタイムの設定
  1. メニューバーの [ランタイム] - [ランタイムのタイプの変更] をクリック
  1. ハードウェア アクセラレータ で `GUP` を設定する
1. 音声ファイルの準備
  1. 📁アイコンをクリックしてファイルの一覧を表示
  1. テキストを抽出したい 音声ファイル を drag & drop で追加
  1. 追加した 音声ファイル を右クリックして `パスをコピー` でファイルの絶対パスをコピーする
  1. `audioPath`: 音声ファイル の絶対パスをセット
1. そのほか設定
  1. `outputCsvName`: 出力するcsvのファイル名をセット
  1. `model`: whisper のモデルを選択する
1. `Ctrl + F9` ですべてのセルを実行する

### 変数のセットとパッケージの導入

In [1]:
#@title 変数のセット
#@markdown 音声ファイルの絶対パス:
audioPath = "/content/audio.mp3" #@param {type:"string"}

#@markdown 出力するCSVのファイル名（/content/ 以下に出力される）:
outputCsvName = "output.csv" #@param {type:"string"}

#@markdown whisper のモデル:
model = "large" #@param ["tiny", "base", "small", "medium", "large"]

import os
outputCsvPaht = os.path.join('/content/', outputCsvName)

#### パッケージの導入

In [2]:
! pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-fa15o2k6
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-fa15o2k6
  Resolved https://github.com/openai/whisper.git to commit b5851c6c40e753606765ac45b85b298e3ae9e00d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 音声をテキストに変換

In [3]:
import whisper
model = whisper.load_model(model)
result = model.transcribe(audioPath)
print('言語:' + result["language"])
print(result["text"])

言語:ja
宮沢賢治作 雨にも負けず雨にも負けず 風にも負けず雪にも 夏の暑さにも負けぬ丈夫な体を持ち 欲はなく決して怒らず いつも静かに笑っている一日に玄米四合と味噌と少しの 野菜を食べあらゆることを自分を感情に入れ ずによく見聞きしわかりそして忘れず野原の松の林の陰の小さなかや ぶきの小屋にいて東に病気の子供あれば 行って看病してやり西に疲れた母あれば 行ってその稲の束を追い南に死にそうな人あれば 行って怖がらなくてもいいといい北に喧嘩や訴訟があれば つまらないからやめろといい緑の時は涙を流し 寒さの夏はおろおろ歩きみんなにデクノボーと呼ばれ 褒められもせず苦にもされずそういうものに私はなりたい南無変行菩薩 南常行菩薩南多方如来 南妙法蓮華経南釈迦無二仏 南常行菩薩南安隆行菩薩


## CSV化

In [4]:
import pandas as pd
audioDf = pd.DataFrame.from_dict(result["segments"])

# csv ファイルに出力
audioDf.to_csv(outputCsvPaht)

# 内容の表示
audioDf

,id,seek,start,end,text,tokens,temperature,avg_logprob,compression_ratio,no_speech_prob
0,0,0,0.00,11.00,宮沢賢治作 雨にも負けず,"[50364, 39194, 3308, 95, 13352, 95, 30118, 119...",0.0,-0.212735,1.405797,0.208469
1,1,0,11.00,14.08,雨にも負けず 風にも負けず,"[50914, 35339, 4108, 4801, 11561, 254, 7625, 1...",0.0,-0.212735,1.405797,0.208469
2,2,0,14.08,19.52,雪にも 夏の暑さにも負けぬ,"[51068, 47939, 4108, 4801, 220, 42708, 2972, 2...",0.0,-0.212735,1.405797,0.208469
3,3,0,19.52,24.38,丈夫な体を持ち 欲はなく,"[51340, 39642, 3203, 29485, 5998, 17694, 6574,...",0.0,-0.212735,1.405797,0.208469
4,4,0,24.38,28.20,決して怒らず いつも静かに笑っている,"[51583, 33540, 8822, 3757, 240, 5154, 18216, 3...",0.0,-0.212735,1.405797,0.208469
5,5,2820,28.20,37.82,一日に玄米四合と味噌と少しの 野菜を食べ,"[50364, 2257, 6890, 4108, 8051, 226, 27742, 19...",0.0,-0.267511,1.261364,0.012428
6,6,2820,37.82,42.76,あらゆることを自分を感情に入れ ずに,"[50845, 3590, 5154, 34072, 4895, 13235, 5998, ...",0.0,-0.267511,1.261364,0.012428
7,7,2820,42.76,48.20,よく見聞きしわかりそして忘れず,"[51092, 5591, 6134, 10470, 25468, 7016, 2849, ...",0.0,-0.267511,1.261364,0.012428
8,8,2820,48.20,55.36,野原の松の林の陰の小さなかや ぶきの小屋にいて,"[51364, 37178, 19683, 2972, 36897, 2972, 32063...",0.0,-0.267511,1.261364,0.012428
9,9,5536,55.36,62.00,東に病気の子供あれば 行って看病してやり,"[50364, 18413, 4108, 30986, 25870, 2972, 7626,...",0.0,-0.166693,1.539773,0.114267
